In [ ]:
import numpy as np
import librosa as lba
import pyloudnorm as pyln

tone_duration = 200
fade_duration = 10

# Split

In [ ]:
audio, sr = lba.load('../stimuli/tones/tones.wav', sr=None, mono=True)
inds = lba.effects.split(audio)
print(inds / sr)

In [ ]:
audio, sr = lba.load('../stimuli/tones/tones.wav', sr=None, mono=True)

inds = lba.effects.split(audio)
tone1 = audio[inds[0, 0]:inds[0, 1]]
tone2 = audio[inds[1, 0]:inds[1, 1]]
tone3 = audio[inds[2, 0]:inds[2, 1]]
tone4 = audio[inds[3, 0]:inds[3, 1]]
tone5 = audio[inds[5, 0]:inds[5, 1]]
tone6 = audio[inds[6, 0]:inds[6, 1]]
tone7 = audio[inds[9, 0]:inds[9, 1]]
midtone = audio[inds[13, 0]:inds[13, 1]]

stim_length = int(tone_duration * sr / 1000.)
fade_length = int(fade_duration * sr / 1000.)
threshold = .001
for i, tone in enumerate((tone1, tone2, tone3, tone4, tone5, tone6, tone7)):
    start = np.where(np.abs(tone) > threshold)[0][0]
    tone = np.copy(tone[start:start+stim_length])
    tone[-fade_length:] *= np.linspace(1, 0, fade_length)
    lba.output.write_wav('../stimuli/tones/pianoA%i.wav' % (i+1), tone, sr)

start = np.where(np.abs(midtone) > threshold)[0][0]
midtone = midtone[start:start+stim_length]
midtone[-fade_length:] *= np.linspace(1, 0, fade_length)
lba.output.write_wav('../stimuli/tones/pianoD#5.wav', midtone, sr)

# Balancing

In [ ]:
for octave in range(1, 8):
    print(octave)
    tone, sr = lba.load('../stimuli/tones/pianoA%i.wav' % octave, sr=None)
    meter = pyln.Meter(sr, block_size=.001, filter_class="K-weighting")
    loudness = meter.integrated_loudness(tone)
    for lkfs in (-13, -16, -19):
        normed_tone = pyln.normalize.loudness(tone, loudness, lkfs)
        lba.output.write_wav('../stimuli/tones/normA%i%slkfs.wav' % (octave, lkfs), normed_tone, sr)

print('Midtone')
tone, sr = lba.load('../stimuli/tones/pianoD#5.wav', sr=None)
meter = pyln.Meter(sr, block_size=.001, filter_class="K-weighting")
loudness = meter.integrated_loudness(tone)
for lkfs in (-13, -16, -19):
    normed_tone = pyln.normalize.loudness(tone, loudness, lkfs)
    lba.output.write_wav('../stimuli/tones/normD#5%slkfs.wav' % lkfs, normed_tone, sr)

# Sequence Creation

In [ ]:
### Main Sequences ###
lkfs = [-13, -16, -19]
octaves = [2, 3, 4, 5, 6, 7]
iois = [1000, 918, 843,
        774, 710, 652,
        599, 550, 504,
        463, 425, 390,
        358, 329, 302]

for lu in lkfs:
    for octave in octaves:
        # Load tone of current octave
        tone, sr = lba.load('../stimuli/tones/normA%i%ilkfs.wav' % (octave, lu), sr=None)
        for ioi in iois:

            # Create array of appropriate length to hold audio sequence
            ms_ioi = ioi / 1000.
            sequence = np.zeros(int(np.ceil(8 * ms_ioi * sr)), dtype=np.float32)

            # Insert tones at appropriate locations
            for i in range(5):
                tone_start = int(np.ceil(i * ms_ioi * sr))
                tone_end = tone_start + tone.shape[0]
                sequence[tone_start:tone_end] = tone

            # Cut silence from the end of the sequence
            sequence = np.trim_zeros(sequence, 'b')

            # Save sequence to WAV file
            lba.output.write_wav('../stimuli/sequence_%i_%i_%i.wav' % (octave, ioi, abs(lu)), sequence, sr)

In [ ]:
### Practice Trials ###
tone, sr = lba.load('../stimuli/tones/normD#5-16lkfs.wav', sr=None)
for ioi in (407, 550, 741):
    # Create array of appropriate length to hold audio sequence
    ms_ioi = ioi / 1000.
    sequence = np.zeros(int(np.ceil(8 * ms_ioi * sr)), dtype=np.float32)

    # Insert tones at appropriate locations
    for i in range(5):
        tone_start = int(np.ceil(i * ms_ioi * sr))
        tone_end = tone_start + tone.shape[0]
        sequence[tone_start:tone_end] = tone

    # Cut silence from the end of the sequence
    sequence = np.trim_zeros(sequence, 'b')

    # Save sequence to WAV file
    lba.output.write_wav('../stimuli/practice_sequence_%i.wav' % ioi, sequence, sr)

In [ ]:
### Metronome ###
tone, sr = lba.load('../stimuli/tones/tick.wav', sr=None)

# Create array of appropriate length to hold audio sequence
ioi = 550
ms_ioi = ioi / 1000.
sequence = np.zeros(int(np.ceil(8 * ms_ioi * sr)), dtype=np.float32)
        
# Insert tones at appropriate locations
for i in range(5):
    tone_start = int(np.ceil(i * ms_ioi * sr))
    tone_end = tone_start + tone.shape[0]
    sequence[tone_start:tone_end] = tone
        
# Cut silence from the end of the sequence
sequence = np.trim_zeros(sequence, 'b')
        
# Save sequence to WAV file
lba.output.write_wav('../stimuli/reference_sequence.wav', sequence, sr)